In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

from scipy import stats
import pingouin as pg
import statsmodels.api as sm
import statsmodels.formula.api as smf

# Chap 8. 로지스틱 회귀분석과 포아송 회귀분석
- 선형회귀분석 : 결과변수는 연속형 정규분포여야 함
    - 실제로는 결과변수가 범주형, 횟수 일 수 있고, 정규분포를 따르지 않음
    $$\mu_y = \beta_0 + \beta_1x_1 + \beta_2x_2 + \dots + \beta_mx_m$$
    - 결과변수의 조건부 평균은 예측변수들의 선형결합. 예측변수가 주어졌ㅇ르 때 결과변수 $\hat{y}$를 추정(예측변수들이 사전에 주어졌을 때 y 분포의 평균
- 일반선형모델 : 정규분포를 따르지 않는 결과변수를 수용할 수 있는 일반화된 회귀 모델
$$f(\mu_y) = \beta_0 + \beta_1x_1 + \dots + \beta_mx_m$$
    - $f(\mu_y)$ : 조건부 평균의 함수(링크함수), 다른 확률분포를 따르는 것으로 가정할 수 있음
    - 링크함수와 확률분포를 지정함으로써 다양한 확장된 선형회귀모델 생성
- 로지스틱 회귀모델
$$ln(\frac{p}{1-p})=\beta_0+\beta_1x_1+\dots+\beta_mx_m$$
    - 결과변수 : 범주형변수 / 확률분포 : 이항분포
- 포아송회귀모델
$$ln(\lambda)=\beta_0+\beta_1x_1+\dots+\beta_mx_m$$
    - 결과변수 : 횟수변수  /  확률분포 : 포아송분포  / $\lambda$: y의 평균
    
## 8-2 이항 로지스틱 회귀분석
- 종속변수가 이분형 범주일 때 독립변수로부터 결과변수 범주 예측
$$ln(\frac{p}{1-p})=\beta_0+\beta_1x_1+\dots+\beta_mx_m$$
    - 특정 사건이 발생할 확률을 직접 추정 : 결과 변수 예측값은 항상 [0,1], 기준값 기준 대소 비교
    - 결과변수 : 범주형변수 / 확률분포 : 이항분포
$$사건발생확률\ p=\frac{e^z}{1+e^z}=\frac{1}{1+e^{-z}},\quad z=\beta_0+\beta_1x_1+\dots+\beta_mx_m$$
    - 예측변수와 결과변수는 S자의 비선형관계를 가지며, p=0.5에서 기울기가 급해짐
- 로그오즈
    - 오즈 :$\frac{p}{1-p}$, 사건이 발생할 가능성이 그렇지 않을 가능성의 배수
    - 로지스틱 회귀모델은 로그오즈에 대한 선형회귀모델 정의
- 오즈비 : 독립변수 한 단위 증가에 따른 오즈(사건발생확률 대 미발생확률 비율)의 변화비율
    $오즈=\frac{P(사건발생)}{P(사건미발생)} = e^z = e^{\beta_0+\sum \beta_ix_i}$
    - 계수가 3인 경우, 2가 증가하면 3^2=9배 확률 증가
    
### 로지스틱 회귀모델의 유의성 검정    
- deviance : 이탈 => 모델의 비 적합도, 값이 작으면 모델이 데이터를 잘 설명한다는 것
    - residual deviance : 예측변수가 모두 포함 된 모델의 이탈도 => 충분한 이탈도의 감소인지 검정 => 카이스퀘어 검정
    - null deviance와 residual deviance의 차이가 0인지 검정, 두 이탈도의 차이가 0이 아니라면 예측변수가 추가된 현 모델이 데이터를 잘 적합시키며 통계적으로 유의한 모델이라는 결론
    - null & residual deviance의 차이는 null model과 residual model의 모수 개수 차이를 자유도로 갖는 카이제곱분포 따름
        - 관측값 n, 제안모델 예측변수 p : df_null = n-1. / df_resid = n-(p+1)
    - 두 deviance의 차아ㅣ를 $\chi^2$ 검정통계량으로 사용하여 카이제곱 검정 수행
- 과산포 : 결과변수의 실체 관측된 분산이 이항분포로부터 기대되는 분산보다 더 큰 경우
    => 표준오차를 왜곡시켜 표준오차의 유의성 검증을 부정확하게 함, glm에서 family를 quasibinomial 사용해줘야 함
    - 과산포 확인 : 
    1) 이항분포 로지스틱회귀모델에서 이탈도와 자유도 간 비율 확인 => 1보다 클 경우 과산포 의심
    2) quasi binomimal과 binomial 두 분포 모델의 검정통계량을 chi2 분포상에서 검정
        - H0: '과산포 비율이 1'
        - p가 유의수준을 만족하면, null & residual deviance 차이는 0이 아니며, null model에 비해 residual model이 데이터를 더 잘 설명해준다는 의미
        
## 8.3 페널티 로지스틱 회귀분석
- 너무 많은 변수가 추가될 경우, 페널티 부과        

In [95]:
churn = pd.read_csv("../data/kwak/mlc_churn.csv", index_col=0).reset_index(drop=True)
churn.head()

,state,account_length,area_code,international_plan,voice_mail_plan,number_vmail_messages,total_day_minutes,total_day_calls,total_day_charge,total_eve_minutes,total_eve_calls,total_eve_charge,total_night_minutes,total_night_calls,total_night_charge,total_intl_minutes,total_intl_calls,total_intl_charge,number_customer_service_calls,churn
0,KS,128,area_code_415,no,yes,25,265.10000,110,45.07000,197.40000,99,16.78000,244.70000,91,11.01000,10.00000,3,2.70000,1,no
1,OH,107,area_code_415,no,yes,26,161.60000,123,27.47000,195.50000,103,16.62000,254.40000,103,11.45000,13.70000,3,3.70000,1,no
2,NJ,137,area_code_415,no,no,0,243.40000,114,41.38000,121.20000,110,10.30000,162.60000,104,7.32000,12.20000,5,3.29000,0,no
3,OH,84,area_code_408,yes,no,0,299.40000,71,50.90000,61.90000,88,5.26000,196.90000,89,8.86000,6.60000,7,1.78000,2,no
4,OK,75,area_code_415,yes,no,0,166.70000,113,28.34000,148.30000,122,12.61000,186.90000,121,8.41000,10.10000,3,2.73000,3,no


In [96]:
churn = churn.drop(['state','area_code'], axis=1)
churn['churn'] = churn['churn'].map(lambda x: 0 if x=='no' else 1)

churn_train = churn.iloc[:3333,:]
churn_test = churn.iloc[3333:, :]

display(churn_train['churn'].value_counts().values / len(churn_train))

array([0.85508551, 0.14491449])

In [97]:
# churn_train['churn'] = churn_train['churn'].map(lambda x: 0 if x==1 else 1)
formula = 'churn ~ account_length'
for i in range(1, len(churn.columns)-1):
    formula = formula + ' + ' + churn.columns[i]
model = smf.glm(formula = formula, data=churn_train, family=sm.families.Binomial()).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                  churn   No. Observations:                 3333
Model:                            GLM   Df Residuals:                     3315
Model Family:                Binomial   Df Model:                           17
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -1079.4
Date:                Sun, 25 Aug 2024   Deviance:                       2158.7
Time:                        23:53:24   Pearson chi2:                 3.45e+03
No. Iterations:                     6   Pseudo R-squ. (CS):             0.1646
Covariance Type:            nonrobust                                         
=================================================================================================
                                    coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------------
Intercept                        -8.6516      0.724    -11.944      0.000     -10.071      -7.232
international_plan[T.yes]         2.0428      0.145     14.040      0.000       1.758       2.328
voice_mail_plan[T.yes]           -2.0250      0.574     -3.527      0.000      -3.150      -0.900
account_length                    0.0008      0.001      0.608      0.543      -0.002       0.004
number_vmail_messages             0.0359      0.018      1.992      0.046       0.001       0.071
total_day_minutes                -0.2442      3.274     -0.075      0.941      -6.662       6.173
total_day_calls                   0.0032      0.003      1.158      0.247      -0.002       0.009
total_day_charge                  1.5127     19.260      0.079      0.937     -36.237      39.262
total_eve_minutes                 0.8187      1.636      0.501      0.617      -2.387       4.025
total_eve_calls                   0.0011      0.003      0.380      0.704      -0.004       0.007
total_eve_charge                 -9.5464     19.244     -0.496      0.620     -47.263      28.171
total_night_minutes              -0.1238      0.876     -0.141      0.888      -1.842       1.594
total_night_calls                 0.0007      0.003      0.246      0.806      -0.005       0.006
total_night_charge                2.8338     19.477      0.145      0.884     -35.340      41.008
total_intl_minutes               -4.3378      5.301     -0.818      0.413     -14.728       6.052
total_intl_calls                 -0.0930      0.025     -3.710      0.000      -0.142      -0.044
total_intl_charge                16.3900     19.632      0.835      0.404     -22.089      54.869
number_customer_service_calls     0.5136      0.039     13.079      0.000       0.437       0.591
=================================================================================================
"""

In [98]:
# 오즈비 구하기
## total_day_charge가 2 증가시 4.539^2배 증가
np.set_printoptions(suppress=True)
pd.options.display.float_format = '{:.5f}'.format
np.exp(model.params)

Intercept                              0.00017
international_plan[T.yes]              7.71182
voice_mail_plan[T.yes]                 0.13199
account_length                         1.00085
number_vmail_messages                  1.03653
total_day_minutes                      0.78333
total_day_calls                        1.00320
total_day_charge                       4.53901
total_eve_minutes                      2.26754
total_eve_calls                        1.00106
total_eve_charge                       0.00007
total_night_minutes                    0.88353
total_night_calls                      1.00070
total_night_charge                    17.01012
total_intl_minutes                     0.01307
total_intl_calls                       0.91122
total_intl_charge               13125028.74056
number_customer_service_calls          1.67124
dtype: float64

In [99]:
# 모델 적합도 검정 : deviance에 대한 chi-square 검정 (lower.tail=False)
1-stats.chi2.cdf(model.null_deviance - model.deviance, df=model.nobs-1-model.df_resid)

0.0

In [100]:
preds = model.predict(churn_test.drop('churn', axis=1)).map(lambda x: 1 if x > 0.5 else 0)
preds.value_counts()

0    1595
1      72
dtype: int64

In [105]:
from sklearn.metrics import confusion_matrix, accuracy_score

print(confusion_matrix(churn_test['churn'], preds))
print(accuracy_score(churn_test['churn'], preds))

[[1414   29]
 [ 181   43]]
0.8740251949610078


In [133]:
model.pvalues <= 0.05

Intercept                       0.00000
international_plan[T.yes]       0.00000
voice_mail_plan[T.yes]          0.00042
account_length                  0.54320
number_vmail_messages           0.04635
total_day_minutes               0.94055
total_day_calls                 0.24705
total_day_charge                0.93740
total_eve_minutes               0.61672
total_eve_calls                 0.70382
total_eve_charge                0.61984
total_night_minutes             0.88765
total_night_calls               0.80563
total_night_charge              0.88432
total_intl_minutes              0.41319
total_intl_calls                0.00021
total_intl_charge               0.40380
number_customer_service_calls   0.00000
dtype: float64

In [149]:
# p-val 상 유의한 회귀계수만 선정 or stepwise
new_model = smf.glm(formula = 'churn ~ international_plan + voice_mail_plan + number_vmail_messages + total_day_charge + total_eve_minutes + total_night_charge + total_intl_calls + total_intl_charge + number_customer_service_calls', data=churn_train, family=sm.families.Binomial()).fit()
new_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                  churn   No. Observations:                 3333
Model:                            GLM   Df Residuals:                     3323
Model Family:                Binomial   Df Model:                            9
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -1080.8
Date:                Mon, 26 Aug 2024   Deviance:                       2161.6
Time:                        06:29:17   Pearson chi2:                 3.44e+03
No. Iterations:                     6   Pseudo R-squ. (CS):             0.1639
Covariance Type:            nonrobust                                         
=================================================================================================
                                    coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------------
Intercept                        -8.0672      0.516    -15.638      0.000      -9.078      -7.056
international_plan[T.yes]         2.0403      0.145     14.048      0.000       1.756       2.325
voice_mail_plan[T.yes]           -2.0032      0.572     -3.500      0.000      -3.125      -0.881
number_vmail_messages             0.0353      0.018      1.963      0.050    5.33e-05       0.070
total_day_charge                  0.0766      0.006     12.022      0.000       0.064       0.089
total_eve_minutes                 0.0072      0.001      6.290      0.000       0.005       0.009
total_night_charge                0.0825      0.025      3.348      0.001       0.034       0.131
total_intl_calls                 -0.0922      0.025     -3.689      0.000      -0.141      -0.043
total_intl_charge                 0.3261      0.075      4.322      0.000       0.178       0.474
number_customer_service_calls     0.5123      0.039     13.087      0.000       0.436       0.589
=================================================================================================
"""

In [164]:
# 다른 변수 고정한 채, 특정 변수만 변화시키면서 사건발생확률에 미치는 영향 확인 가능
temp = churn['number_customer_service_calls'].value_counts().index.to_frame().rename(columns={0:"number_customer_service_calls"})
fix_cat_cols = ['international_plan', 'voice_mail_plan', ]
fix_num_cols = ['number_vmail_messages', 'total_day_charge', 'total_eve_minutes', 'total_night_charge', 'total_intl_calls', 'total_intl_charge']
# churn_copy = churn.copy()[fix_cat_cols + fix_num_cols]

for col in fix_cat_cols:
    temp[col] = 'no'
for col in fix_num_cols:
    temp[col] = churn_copy[col].mean()

temp['prob'] = new_model.predict(temp)
temp # 고객이탈확률은 전화횟수가 늘어남에 따라 증가함을 알 수 있음

,number_customer_service_calls,international_plan,voice_mail_plan,number_vmail_messages,total_day_charge,total_eve_minutes,total_night_charge,total_intl_calls,total_intl_charge,prob
1,1,no,no,7.75520,30.64967,200.63656,9.01773,4.43520,2.77120,0.09504
2,2,no,no,7.75520,30.64967,200.63656,9.01773,4.43520,2.77120,0.14914
0,0,no,no,7.75520,30.64967,200.63656,9.01773,4.43520,2.77120,0.05920
3,3,no,no,7.75520,30.64967,200.63656,9.01773,4.43520,2.77120,0.22634
4,4,no,no,7.75520,30.64967,200.63656,9.01773,4.43520,2.77120,0.32809
5,5,no,no,7.75520,30.64967,200.63656,9.01773,4.43520,2.77120,0.44904
6,6,no,no,7.75520,30.64967,200.63656,9.01773,4.43520,2.77120,0.57632
7,7,no,no,7.75520,30.64967,200.63656,9.01773,4.43520,2.77120,0.69423
9,9,no,no,7.75520,30.64967,200.63656,9.01773,4.43520,2.77120,0.86348
8,8,no,no,7.75520,30.64967,200.63656,9.01773,4.43520,2.77120,0.79121


In [167]:
# 과산포 확인
## 1) 비율검정
print(new_model.deviance / new_model.df_resid)

## 2) 카이제곱 검정
quasi_bin_model = smf.glm(formula = 'churn ~ international_plan + voice_mail_plan + number_vmail_messages + total_day_charge + total_eve_minutes + total_night_charge + total_intl_calls + total_intl_charge + number_customer_service_calls', data=churn_train, family=sm.families.Binomial()).fit(scale='X2')
1-stats.chi2.cdf(quasi_bin_model.scale * new_model.df_resid, new_model.df_resid)

0.6505038485009739


In [174]:
1-stats.chi2.cdf(quasi_bin_model.scale * new_model.df_resid, new_model.df_resid)

0.08385587236163594

In [198]:
# 페널티 로지스틱 회귀분석
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import mean_squared_error

pima = pd.read_csv("../data/kwak/diabetes.csv").rename(columns={'SkinThickness':'triceps', 'BMI':'mass'})
pima = pima.dropna(axis=0)

X_train, X_test, y_train, y_test = train_test_split(pima.drop('Outcome', axis=1), pima['Outcome'], stratify=pima['Outcome'], test_size=0.3, shuffle=True, random_state=1)

cv_result = {}
kf = KFold(n_splits=5, shuffle=True, random_state=123)
for i in np.arange(0, 1, 0.1):
    for tr_idx, val_idx in kf.split(X_train, y_train):
        X_tr, y_tr = X_train.iloc[tr_idx], y_train.iloc[tr_idx]
        X_val, y_val = X_train.iloc[val_idx], y_train.iloc[val_idx]    
        
        # l1_wt = 0(ridge), 1(lasso)
        model = sm.GLM(endog=y_tr, exog=X_tr, family=sm.families.Binomial()).fit_regularized(method='elastic_net', alpha=i, L1_wt=1)
        preds = model.predict(X_val)
        
        score = np.sqrt(mean_squared_error(y_val, preds))
    
    cv_result[i] = np.mean(score)

cv_result = pd.DataFrame.from_dict(cv_result, orient='index').rename(columns={0:'scores'})
cv_result.sort_values('scores', ascending=True)

,scores
0.00000,0.43484
0.10000,0.43633
0.20000,0.43901
0.30000,0.44303
0.40000,0.44364
0.50000,0.44468
0.80000,0.44863
0.90000,0.45011
0.60000,0.45878
0.70000,0.46039


In [202]:
# lasso 0
from sklearn.metrics import confusion_matrix, accuracy_score

cv_log_model = sm.GLM(endog=y_train, exog=X_train, family=sm.families.Binomial()).fit_regularized(method='elastic_net', alpha=0, L1_wt=1)
log_preds = cv_log_model.predict(X_test).map(lambda x: 1 if x > 0.5 else 0)

print(confusion_matrix(y_test, log_preds))
print(accuracy_score(y_test, log_preds))

[[128  22]
 [ 58  23]]
0.6536796536796536


In [203]:
# comparize with ref model
temp_model = sm.GLM(endog=y_train, exog=X_train, family=sm.families.Binomial()).fit()
temp_preds = temp_model.predict(X_test).map(lambda x: 1 if x > 0.5 else 0)

print(confusion_matrix(y_test, temp_preds))
print(accuracy_score(y_test, temp_preds))

[[127  23]
 [ 58  23]]
0.6493506493506493


## 8.4 다항 로지스틱회귀분석
- 예측변수로부터 세 개 이상의 사건(범주)을 갖는 결과변수의 사건발생확률 예측 => 새로운 케이스의 소속범주 예측
$$ln(\frac{p_k}{p_1})=\beta_{0k}+\beta_{1k}x_1+\dots+\beta_{mk}x_m,\quad k=2,\dots,g \\ k:사건,\ p_k:사건k\ 발생확률,\ m:예측변수\ 갯수$$
- g개의 범주가 있을 때, 총 (g-1)개의 다항 로지스틱 회귀모델이 생성되며, 각 범주당 하나씩 기준범주에 대한 로그오즈를 설명하는 회귀모델로 정의
- 각 범주별 발생확률 $p_k=P[y=k]=\frac{e^{z_k}}{1+\sum^g_{h=2}e^{z_h}},\ k=2,3,\dots,g \\ z_k=\beta_{0k}+\beta_{1k}x_1+\dots+\beta_{mk}x_m,\ k=2,3,\dots,g$
- 기준범주 발생확률 $p_1=P[y=1]=\frac{1}{1+\sum^g_{h=2}e^{z_h}}$

In [254]:
from sklearn.datasets import load_iris

iris = load_iris()
X = pd.DataFrame(iris['data'], columns=list(map(lambda x : x.replace(' (cm)', ''), iris['feature_names'])))
y = pd.DataFrame(iris['target'], columns=['species'])
iris = pd.concat([X, y], axis=1)
iris.head()

,sepal length,sepal width,petal length,petal width,species
0,5.10000,3.50000,1.40000,0.20000,0
1,4.90000,3.00000,1.40000,0.20000,0
2,4.70000,3.20000,1.30000,0.20000,0
3,4.60000,3.10000,1.50000,0.20000,0
4,5.00000,3.60000,1.40000,0.20000,0


In [262]:
# newton 기본 옵티마이져는 Hessian이 singular가 되면 fail 
X = sm.add_constant(X)
mlog_model = sm.MNLogit(y, X).fit(method='bfgs')
mlog_model.summary()

         Current function value: 0.057112
         Iterations: 35
         Function evaluations: 37
         Gradient evaluations: 37


<class 'statsmodels.iolib.summary.Summary'>
"""
                          MNLogit Regression Results                          
==============================================================================
Dep. Variable:                species   No. Observations:                  150
Model:                        MNLogit   Df Residuals:                      140
Method:                           MLE   Df Model:                            8
Date:                Tue, 10 Sep 2024   Pseudo R-squ.:                  0.9480
Time:                        07:38:19   Log-Likelihood:                -8.5668
converged:                      False   LL-Null:                       -164.79
Covariance Type:            nonrobust   LLR p-value:                 9.200e-63
================================================================================
   species=1       coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
const            1.6444   1550.515      0.001      0.999   -3037.309    3040.598
sepal length    -1.4959    444.817     -0.003      0.997    -873.321     870.330
sepal width     -8.0560    282.766     -0.028      0.977    -562.267     546.155
petal length    11.9301    374.116      0.032      0.975    -721.324     745.184
petal width      1.7039    759.366      0.002      0.998   -1486.627    1490.035
--------------------------------------------------------------------------------
   species=2       coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
const           -6.5053   1550.533     -0.004      0.997   -3045.494    3032.483
sepal length    -8.0348    444.835     -0.018      0.986    -879.896     863.827
sepal width    -15.8195    282.793     -0.056      0.955    -570.083     538.444
petal length    22.1797    374.155      0.059      0.953    -711.152     755.511
petal width     14.0603    759.384      0.019      0.985   -1474.304    1502.425
================================================================================
"""

In [268]:
np.exp(mlog_model.params)

,0,1
const,5.17777,0.00150
sepal length,0.22405,0.00032
sepal width,0.00032,0.00000
petal length,151772.59580,4290544646.15967
petal width,5.49538,1277374.35827


In [267]:
mlog_model.params

,0,1
const,1.64437,-6.50528
sepal length,-1.49590,-8.03479
sepal width,-8.05603,-15.81951
petal length,11.93014,22.17968
petal width,1.70391,14.06032


In [266]:
np.exp(mlog_model.params)

,0,1
const,5.17777,0.00150
sepal length,0.22405,0.00032
sepal width,0.00032,0.00000
petal length,151772.59580,4290544646.15967
petal width,5.49538,1277374.35827
